Modelado clásico de lenguaje natural
===============================

## Creando un pipeline de preprocesamiento de texto

A pesar de que los métodos anteriores son no supervisados, son de utilidad para el modelado de de problemas no supervisados como supervisados. Para llevar estos métodos a un entorno práctico normalmente se construyen flujos de procesamiento como el que se muestra más abajo. Estos flujos se los llama Pipeline:

<img src='https://github.com/santiagxf/M72109/blob/master/NLP/Docs/atap_0406.png?raw=1' />

A modo de ejemplo, vamos a utilizar la API de Scikit-Learn para generar cada uno de estos pasos y así construir un modelo que resuelva un problema de negocio de punta a punta.

**¿Que es lo que vamos a hacer?**
Intentaremos construir un pipeline de machine learning donde como entrada recibamos texto, ejecutemos todos los pasos que vimos en este notebook incluyendo:
 - Eliminación de stopwords
 - Tokenización
 - Stemming y Lemmatization
 - Procesamiento especico del tema
 - Creación de features utilizando algun metodo de reducción de dimensionalidad, SVD, LSI, LDA

, para luego utilizar estas features para entrenar un modelo que nos permita predecir alguna propiedad interesante del set de datos. En este caso en particular, donde estamos viendo tweets, algunos casos interesantes podrían ser:
 - Predecir el sector al que pertenece el tweet: Alimentación, Bebidas, etc.
 - Predecir el paso en el Marketing Funel al que pertece

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/
    
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/modeling/classic-modeling.txt \
    --quiet --no-clobber
!pip install -r classic-modeling.txt

Primero importaremos algunas librerias necesarias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Sobre el set de datos con el que vamos a trabajar

Utilizaremos como ejemplo un set de datos en español que contiene tweets que diferentes usuarios han publicado en relación a diferentes marcas de productos u empresas en el rubro de alimentación, construcción, automoviles, etc. Estos tweets, a su vez, están asociados a una de las diferentes fases en el proceso de ventas (también conocido como Marketing Funel) y por eso están tagueados con las fases de:
 - Awareness – el cliente es conciente de la existencia de un producto o servicio
 - Interest – activamente expresa el interes de un producto o servicio
 - Evaluation – aspira una marca o producto en particular
 - Purchase – toma el siguiente paso necesario para comprar el producto o servicio
 - Postpurchase - realización del proceso de compra. El cliente compara la diferencia entre lo que deseaba y lo que obtuvo

Referencia: [Spanish Corpus of Tweets for Marketing](http://ceur-ws.org/Vol-2111/paper1.pdf

> Nota: La version de este conjunto de datos que utilizaremos aqui es una versión preprocesada del original.

In [3]:
tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

Inspeccionamos el set de datos

In [4]:
tweets.head(5)

,TEXTO,SECTOR,MARCA,CANAL,AWARENESS,EVALUATION,PURCHASE,POSTPURCHASE,NC2
0,#tablondeanuncios Funda nordica ikea #madrid h...,RETAIL,IKEA,Microblog,0,0,0.0,0,1.0
1,#tr Me ofrezco para montar muebles de Ikea - H...,RETAIL,IKEA,Microblog,0,0,0.0,0,1.0
2,#VozPópuli Vozpópuli @voz_populi - #LoMásLeido...,RETAIL,ALCAMPO,Microblog,0,0,0.0,0,1.0
3,#ZonaTecno Destacado: Todo lo que hay que sabe...,RETAIL,CARREFOUR,Microblog,0,0,0.0,0,1.0
4,$Carrefour retira pez #Panga. OCU y grupos x #...,RETAIL,CARREFOUR,Microblog,0,0,0.0,0,1.0


In [5]:
tweets.groupby('SECTOR').head(1)[['TEXTO', 'SECTOR']]

,TEXTO,SECTOR
0,#tablondeanuncios Funda nordica ikea #madrid h...,RETAIL
725,"""Ilcinsisti lis MB dispiniblis"" te odeeeeeo Mo...",TELCO
964,#CarlosSlim y Bimbo lanzarán un vehículo eléct...,ALIMENTACION
1298,"‼🏎Toyota #Day, 4ruedas ,1/4 milla, 1 #pasión, ...",AUTOMOCION
1748,"""- Tú qué.\n- Yo na.""\nConversaciones banco sa...",BANCA
2348,"- Cariño, te juro que sólo tenían Cruzcampo en...",BEBIDAS
3023,#adidas #hockey Amenabar 2080 CABA https://t.c...,DEPORTES


## Creando un paso de Pipeline para procesamiento de texto

El paso más complejo que tenemos para crear es quizas el preprocesamiento del texto. Esto lo podemos encapsular en un modulo de Scikit-Learn. Esta libreria tiene 2 tipos de modulos:
 - Transformers
 - Estimators

Los transformers toman un set de features y devuelven otro set de features, por eso es que reciben el nombre de "trasformers", porque basicamente transforman vectores. Los estimators, por el contrario, reciben un set de features y producen un podelo que aproxima, o estima, una variable target. Por este motivo, estos modulos reciben el nombre de "estimators".

¿Qué modulo les parece que va a implementar nuestro proceso de "preprocesamiento" de texto?

In [6]:
import unidecode
import spacy
import es_core_news_sm as spa
import re
import sklearn
import nltk
from nltk import stem
from nltk.corpus import stopwords
from nltk.tokenize.casual import TweetTokenizer

class TextNormalizer(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, language='spanish'):
        nltk.download('stopwords')

        self.parser = spa.load() # Cargamos el parser en español
        self.tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) # Creamos un tokenizer
        self.stemmer = stem.SnowballStemmer(language=language) # Creamos un steammer
        self.lemmatizer = lambda word : " ".join([token.lemma_ for token in self.parser(word)]) # Creamos un lemmatizer
        self.stopwords = set(stopwords.words(language)) # Instanciamos las stopwords en español
        self.urls_regex = re.compile('http\S+') # Usamos una expresion regular para encontrar las URLs
    
    def process_text(self, text):
        tokens = self.tokenizer.tokenize(text)
        tokens = [token for token in tokens if not re.match(self.urls_regex, token)]
        tokens = [token for token in tokens if len(token) > 4]
        tokens = [token for token in tokens if token not in self.stopwords]
        tokens = [unidecode.unidecode(token) for token in tokens] # Quitamos acentos
        tokens = [self.lemmatizer(token) for token in tokens]
        return tokens
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for doc in X:
          yield ' '.join(self.process_text(text=doc))

Instanciamos nuestro preprocesamiento de texto

In [7]:
normalizer = TextNormalizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/santiagxf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Podemos ver como funciona nuestro modulo de preprocesamiento de texto al llamarlo con la función transform:

In [8]:
tweet = tweets['TEXTO'][5]
print(tweet)

. @PoliciadeBurgos @PCivilBurgos @Aytoburgos Mismo peligro c/ Rio Viejo junto Mercadona Villimar


In [9]:
list(normalizer.transform([tweet]))

['Mismo peligrar Viejo juntar Mercadona Villimar']

*Nota: Usamso list() solamente porque transform es una operación de tipo lazy. Esto significa que no se ejecuta hasta que enumeramos los resultados*

### Construyendo nuestro pipeline

Importamos algunas librerias que necesitaremos

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

Instanciamos nuestro vectorizador, en este caso usando el método TF-IDF

In [15]:
vectorizer = TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')

Instanciamos nuestro generador de features, que en este caso son los tópicos que LDA genere

In [16]:
featurizer = LatentDirichletAllocation(n_components=7)

Instanciamos nuestro clasificador que utilizará las features generadas hasta este momento

In [17]:
estimator = LogisticRegression(max_iter=10000, multi_class='multinomial')

Creamos un pipeline que ejecute todos los pasos en secuencia

In [18]:
pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', vectorizer),
                           ('featurizer', featurizer),
                           ('estimator', estimator)])

### Entrenando nuestro pipeline

En este caso intentaremos predecir el sector al que pertenece un tweet en particular. Para ello, como en todo proceso de machine learning separaremos nuestros datos en training y testing, para poder evaluar los resultados:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, stratify=tweets['SECTOR'])

El método fit intrenará nuestro modelo de punta a punta. Tomará unos minutos

In [ ]:
model = pipeline.fit(X=X_train, y=y_train)

Es hora de ver que tan bien le fué a nuestro modelo en esta tarea

In [ ]:
predictions = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

ALIMENTACION       0.00      0.00      0.00       110
  AUTOMOCION       0.00      0.00      0.00       148
       BANCA       0.28      0.11      0.15       198
     BEBIDAS       0.25      0.28      0.26       223
    DEPORTES       0.27      0.30      0.28       216
      RETAIL       0.21      0.52      0.29       268
       TELCO       0.00      0.00      0.00        79

    accuracy                           0.23      1242
   macro avg       0.14      0.17      0.14      1242
weighted avg       0.18      0.23      0.18      1242



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


¿Les parece que estás métricas son buenas? ¿Se les ocurre como mejorarlo? Algunas ideas:
 - ¿Quien funcionará mejor? ¿Stemmer o Lemmatization?
 - ¿Qué será mejor hacer con los hashtags? ¿Quitarlos?
 - ¿Que cantidad de factores latentes funcionará mejor? ¿7, 10, 200, 300?
 - ¿Es Logistic Regression el mejor clasificador que podemos probar? ¿Si subimos la cantidad de tópicos que me sería mejor utilizar?